In [1]:
# Read RData
from __future__ import unicode_literals
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import json
from os import listdir, getcwd
from os.path import isfile, join
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri
import numpy as np

# Read all files in a directory
mypath = getcwd()+"/data/tweet_youtube"
onlyfiles = [ join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
tweets_time = {}
for f in onlyfiles:
    ro.r['load'](f,ro.globalenv)

shares = ro.globalenv[str('data')].rx2("numShare")
views = ro.globalenv[str('data')].rx2("dailyViewcount")

In [2]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
from __future__ import unicode_literals

%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.contrib import learn
from sklearn.metrics import mean_squared_error
from lstm import generate_data, lstm_model, load_csvdata
import dateutil.parser
import datetime
import matplotlib.dates as mdates

In [3]:
LOG_DIR = './ops_logs/lstm_popularity'
TIMESTEPS = 30
RNN_LAYERS = [{'num_units': 30},{'num_units': 60},{'num_units': 60}]
DENSE_LAYERS = [40,30]
TRAINING_STEPS = 10000
BATCH_SIZE = 220
PRINT_STEPS = TRAINING_STEPS / 10

In [4]:
s_i = shares[2][:120]
v_i = views[2][:120]

data = []
for i in range(120):
    data.append([s_i[i]])
    data.append([v_i[i]])
data_vs = pd.DataFrame(data, 
                       columns = ['data'])
X, y = load_csvdata(data_vs, TIMESTEPS, seperate=False)

In [ ]:
regressor = learn.Estimator(model_fn=lstm_model(TIMESTEPS, RNN_LAYERS, DENSE_LAYERS,\
                                               optimizer = "Adagrad"),\
                            model_dir=LOG_DIR)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'save_summary_steps': 100, '_num_ps_replicas': 0, '_task_type': None, '_environment': 'local', '_is_chief': True, 'save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f43af28fc90>, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, 'tf_random_seed': None, 'keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', 'save_checkpoints_steps': None, '_master': '', 'keep_checkpoint_max': 5}


In [ ]:
validation_monitor = learn.monitors.ValidationMonitor(X['val'], y['val'],
                                                     every_n_steps=PRINT_STEPS,
                                                     early_stopping_rounds=100)
regressor.fit(X['train'], y['train'],
              monitors=[validation_monitor],
              batch_size=BATCH_SIZE,
              steps=TRAINING_STEPS)


Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


In [ ]:
#not used in this example but used for seeing deviations
#rmse = np.sqrt(((predicted - y['test']) ** 2).mean(axis=0))
predicted = regressor.predict(X['test'])
pred = []
for i in predicted:
    pred.append(i)
#score = mean_squared_error(p, y['test'])
#print ("MSE: %f" % rmse)

In [ ]:
plot_predicted = plt.plot(pred, label='predicted')
plot_test = plt.plot(y['test'], label='test')
plt.legend(handles=plot_predicted+plot_test)